In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L4\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['Capital.csv', 'Central.csv', 'Dunwoody.csv', 'Genese.csv', 'Hudson.csv', 'LongIsland.csv', 'Milwood.csv', 'NYC.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[3]))

,Demand,Normalized_net,Temprature
0,1042.4,0.517980,-8.1
1,991.2,0.492538,-8.0
2,961.5,0.477780,-7.9
3,947.1,0.470625,-7.7
4,951.2,0.472662,-7.7
...,...,...,...
43819,1179.4,0.586057,3.2
43820,1136.7,0.564839,2.6
43821,1094.5,0.543869,1.9
43822,1049.6,0.521558,0.1


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[3]

zoneData = pd.read_csv(os.path.join(path, fileName), usecols= ["Demand", "Temprature"])
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
# zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_Genese,Temp_Genese
0,1042.4,-8.1
1,991.2,-8.0
2,961.5,-7.9
3,947.1,-7.7
4,951.2,-7.7
...,...,...
43819,1179.4,3.2
43820,1136.7,2.6
43821,1094.5,1.9
43822,1049.6,0.1


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_Genese,Temp_Genese,Lag-2,Lag-7
0,1181.6,-17.0,1108.0,1042.4
1,1144.1,-17.3,1078.2,991.2
2,1125.8,-17.4,1065.9,961.5
3,1130.8,-17.4,1064.9,947.1
4,1177.2,-17.3,1091.1,951.2
...,...,...,...,...
43651,1179.4,3.2,1171.5,1119.7
43652,1136.7,2.6,1130.0,1105.0
43653,1094.5,1.9,1078.4,1085.8
43654,1049.6,0.1,1014.3,1046.8


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 18s 12ms/step - loss: 0.0130 - mape: 6556.2500 - mae: 0.0885 - val_loss: 0.0077 - val_mape: 46.6328 - val_mae: 0.0717
Epoch 2/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0083 - mape: 6214.7837 - mae: 0.0704 - val_loss: 0.0078 - val_mape: 47.7305 - val_mae: 0.0697
Epoch 3/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0060 - mape: 5832.1875 - mae: 0.0599 - val_loss: 0.0080 - val_mape: 39.9067 - val_mae: 0.0674
Epoch 4/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0053 - mape: 5541.8516 - mae: 0.0557 - val_loss: 0.0066 - val_mape: 40.6845 - val_mae: 0.0603
Epoch 5/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0047 - mape: 5546.8359 - mae: 0.0523 - val_loss: 0.0075 - val_mape: 33.3134 - val_mae: 0.0650
Epoch 6/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0044 - mape: 4946.5640 - mae: 0.0499 - val_l

1105/1105 [==============================] - 12s 11ms/step - loss: 0.0021 - mape: 3232.9170 - mae: 0.0339 - val_loss: 0.0059 - val_mape: 31.3834 - val_mae: 0.0548
Epoch 48/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0021 - mape: 3023.6624 - mae: 0.0339 - val_loss: 0.0074 - val_mape: 31.6896 - val_mae: 0.0621
Epoch 49/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0020 - mape: 3515.7625 - mae: 0.0334 - val_loss: 0.0068 - val_mape: 32.6421 - val_mae: 0.0577
Epoch 50/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0020 - mape: 3299.2725 - mae: 0.0333 - val_loss: 0.0070 - val_mape: 34.2485 - val_mae: 0.0593
Epoch 51/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0020 - mape: 3249.3823 - mae: 0.0332 - val_loss: 0.0065 - val_mape: 32.6892 - val_mae: 0.0566
Epoch 52/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0019 - mape: 3522.9905 - mae: 0.0326 - val_loss: 0.

1105/1105 [==============================] - 12s 10ms/step - loss: 0.0011 - mape: 2821.5361 - mae: 0.0248 - val_loss: 0.0076 - val_mape: 34.5054 - val_mae: 0.0621
Epoch 94/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0011 - mape: 2733.4543 - mae: 0.0245 - val_loss: 0.0074 - val_mape: 34.5748 - val_mae: 0.0615
Epoch 95/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0011 - mape: 2325.5657 - mae: 0.0245 - val_loss: 0.0077 - val_mape: 35.3272 - val_mae: 0.0630
Epoch 96/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0010 - mape: 2605.2732 - mae: 0.0244 - val_loss: 0.0074 - val_mape: 34.9479 - val_mae: 0.0621
Epoch 97/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0010 - mape: 2515.5488 - mae: 0.0242 - val_loss: 0.0069 - val_mape: 34.2512 - val_mae: 0.0589
Epoch 98/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0010 - mape: 2727.9983 - mae: 0.0241 - val_loss: 0.

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 1201.112779378891


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 6ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.1824688 , 0.17067903, 0.1582426 , ..., 0.38144845, 0.33287492,
       0.29165882], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.29217467, 0.26451668, 0.25029663, ..., 0.34734846, 0.31542445,
       0.29679619])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,1016.9,862.602539
1,978.0,846.020691
2,958.0,828.529236
3,962.4,828.194519
4,998.4,864.586853
...,...,...
4363,1179.4,1231.430908
4364,1136.7,1203.267456
4365,1094.5,1142.460449
4366,1049.6,1074.143433


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

7.256660723123251